<a href="https://colab.research.google.com/github/suhyun6363/ReviewGeneration/blob/main/%EC%9A%94%EA%B8%B0%EC%9A%94_%EB%A6%AC%EB%B7%B0%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EB%A6%AC%EB%B7%B0%EC%83%9D%EC%84%B1%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2024.01.01~2024.06.15 동안 `'치킨', '피자/양식', '중국집', '한식', '일식/돈까스', '야식', '분식'`

음식 카테고리별로 `덕성여자대학교 주변 리뷰 많은 순으로 상위 5개` 가게에 작성된 리뷰 데이터를 모아 `kogpt-2 모델`을 파인튜닝 학습한 리뷰 생성 딥러닝 모델

------------------------------------

## Data   
1. 환경 설정   
  1.1 크롬 및 크롬 드라이버 설치 - `120.0.6099.71` 버전으로 다운그레이드/설치   
  1.2 `Selenium` 설치 - 동적 크롤링 도구   
  1.3 리뷰 데이터 크롤링 URL: [요기요](https://www.yogiyo.co.kr/mobile/#/)
2. 데이터 수집 과정   
  2.1 요기요 웹사이트 접속 및 `덕성여자대학교` 주소 입력   
  2.2 `7종류` 음식 카테고리 선택 및 `리뷰 많은 순` 정렬    
  2.3 음식점 리뷰 데이터 크롤링   
  + 각 카테고리에서 `상위 5개`의 음식점을 선택하고 해당 음식점 페이지 > 리뷰로 이동    
  + 목표 날짜(`2024년 1월 1일`)까지 리뷰를 로드
  + 각 리뷰에서 `맛/양/배달 평점`, `주문 아이템`, `리뷰 텍스트`를 추출   
  2.4 csv 파일로 데이터 저장
3. 데이터 수집 결과   
전처리 전: (17460, 7)   
전처리 후:   

| menu_category        | # of Sentence |
| ------------ | -------------: |
|치킨   |       1260   |
| 피자/양식 |          1470 |
|중국집 | 2200 |
|한식 | 480 |
| 일식/돈까스 | 990 |
| 야식 | 440 |
| 분식 | 1160 |
|  | 8000|

+ 한 가게가 여러 메뉴 카테고리에 속해 있어 메뉴 카테고리마다 중복되는 상호명 제거
+ 특수문자, 이모지, 영어 표현, 숫자 제거
+ 현실감 있는 리뷰 텍스트를 위해 오타, 띄어쓰기는 그대로 두어 형태소 분석, 불용어 제거 처리하지 않음   

## Pretrained model/Tokenizer
`skt/kogpt2` 모델과 토크나이저 이용   
1. Model

| Model       |  # of params |   Type   | # of layers  | # of heads | ffn_dim | hidden_dims |
|--------------|:----:|:-------:|--------:|--------:|--------:|--------------:|
| `kogpt2-base-v2` |  125M  |  Decoder |   12     | 12      | 3072    | 768 |   

2. Tokenizer   
[`tokenizers`](https://github.com/huggingface/tokenizers) 패키지의 `Character BPE tokenizer`로 학습됨

사전 크기는 51,200 이며 대화에 자주 쓰이는 아래와 같은 이모티콘, 이모지 등을 추가하여 해당 토큰의 인식 능력을 올림
> 😀, 😁, 😆, 😅, 🤣, .. , `:-)`, `:)`, `-)`, `(-:`...

```python
> from transformers import PreTrainedTokenizerFast
> tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
> tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")
['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.', '😤', ':)', 'l^o']
```

### 내부 동작 원리 (알고리즘)
```
Class CustomDataset(Dataset):
#  PyTorch의 Dataset 클래스를 상속받아 사용자 정의 데이터셋을 구현
텍스트 데이터를 받아 토큰화하고, 모델 학습에 적합한 입력 데이터로 변환하는 역할
__init__: 데이터셋을 초기화
__len__: 데이터셋의 크기를 반환
__getitem__: 주어진 인덱스에 해당하는 데이터를 토큰화하여 반환
```
+ 토크나이저 및 모델 로드: 지정된 경로에서 토크나이저와 모델을 불러옴
+ 패딩 토큰 설정: 필요한 경우 패딩 토큰을 설정
+ 데이터 로더 설정: 데이터셋과 데이터 로더를 생성
+ 최적화 및 스케줄러 설정: 옵티마이저와 학습 스케줄러를 설정
+ 디바이스 설정 및 모델 학습 모드로 전환: 모델을 적절한 디바이스에 할당하고 학습 모드로 전환
+ 학습 루프: 각 에폭에 대해 손실을 계산하고 모델 파라미터를 업데이트
+ 모델 및 토크나이저 저장: 학습이 완료된 후 모델과 토크나이저를 저장

**주요 하이퍼파라미터**
1. batch_size: 32
2. epochs: 5
3. learning_rate: 5e-5
4. max_length:50 (CustomDataset 클래스에서 설정)
5. num_workers: 2
6. early_stopping_patience:3 (EarlyStoppingCallback에서 설정 가능)

**trian_loss: 0.531**

### 활용 용도
메뉴 리뷰를 자동으로 생성하여 마케팅 자료로 활용할 수 있다. 나아가, 데이터를 배달 음식 리뷰가 아닌 제품, 서비스 리뷰 데이터를 추가하여 학습하면 추후 출시될 신제품, 서비스의 마케팅 자료로 활용하기에 좋다.

+ 적용 사례
  - [네이버 커머스 솔루션마켓 CLOVA 메시지마케팅](https://solution.smartstore.naver.com/ko/solution/2NDiArqyVW4mYsqcXUoXYJ/detail) : 스토어 정보, 쿠폰, 판매 상품, 메시지 소재 등 다양한 정보를 활용하여 HyperCLOVA가 3종의 마케팅 메시지(제목, 내용)를 자동으로 생성
  - [현대백화점의 루이스(Lewis)](https://n.news.naver.com/mnews/article/025/0003262163?sid=101) : 판촉행사 소개문 작성 등 마케팅 문구제작에 특화된 현대백화점의 사내 카피라이팅 시스템'
  - [뤼튼](https://wrtn.ai/): 간단한 키워드만으로 완성도 높은 광고카피, 제품 소개 문구를 작성할 수 있는 AI 카피라이팅 서비스
  - [킵그로우](https://app.keepgrow.com/admin/products/category/65) : 네이버 스마트스토어 기반으로 고객사의 인스타그램 계정에 게시물을 주기적으로 포스팅 해주는 '인스타그램 Ai 피드 포스팅' 기능

[참고자료] [네이버 클라우드 플랫폼 블로글](https://blog.naver.com/n_cloudplatform/222913297671)

## 1. 데이터 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service  # 추가된 import
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException
import os
import json
import re
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException

WAIT_TIME = 1

url = 'https://www.yogiyo.co.kr/mobile/#/'
results = []

# 리뷰 날짜 형식 처리 함수
def parse_review_date(date_text):
    now = datetime.now()
    if '시간 전' in date_text:
        hours = int(date_text.split('시간 전')[0])
        return now - timedelta(hours=hours)
    elif '어제' in date_text:
        return now - timedelta(days=1)
    elif '일주일 전' in date_text:
        return now - timedelta(weeks=1)
    elif '일 전' in date_text:
        days = int(date_text.split('일 전')[0])
        return now - timedelta(days=days)
    else:
        return datetime.strptime(date_text, "%Y년 %m월 %d일")

# 더보기 버튼이 나타날 때까지 기다리는 함수
def wait_for_more_button():
    try:
        btn_more = wait.until(EC.presence_of_element_located((By.XPATH, "//li[@class='list-group-item btn-more']/a")))
        return btn_more
    except:
        return None


# 맛, 양, 배달 정보 추출 함수
def extract_review_info(review):
    # #review > li:nth-child(2) > div:nth-child(2) > div > span.category > span:nth-child(3)#review > li:nth-child(2) > div:nth-child(2) > div > span.category > span:nth-child(3)
    try:
        star_category = review.find_elements(By.CSS_SELECTOR, "span.category > span")
        try:
            taste = star_category[2].text
            if taste == '':
                taste = "N/A"
        except IndexError:
            taste = "N/A"

        try:
            quantity = star_category[5].text
            if quantity == '':
                quantity = "N/A"
        except IndexError:
            quantity = "N/A"

        try:
            delivery = star_category[8].text
            if delivery == '':
                delivery = "N/A"
        except IndexError:
            delivery = "N/A"

        # 주문 아이템 텍스트 추출
        try:
            order_items = review.find_element(By.CSS_SELECTOR, "div.order-items").text
            if order_items == '':
                order_items = "N/A"
        except Exception as e:
            order_items = "N/A"

        try:
            review_text = review.find_element(By.CSS_SELECTOR, "p").text
            if review_text == '':
                review_text = "N/A"
        except Exception as e:
            order_items = "N/A"
        print(taste, quantity, delivery, order_items, review_text)
        return taste, quantity, delivery, order_items, review_text

    except Exception as e:
        print(f"Error extracting review info: {e}")

# 목표 날짜까지 리뷰 로드 함수
def load_reviews_until_target_date(target_date):
    continue_loading = True
    while continue_loading:
        reviews = driver.find_elements(By.XPATH, "//li[@class='list-group-item star-point ng-scope']")
        for review in reviews:
            try:
                review_time = review.find_element(By.CSS_SELECTOR, "span.review-time").text
                review_date = parse_review_date(review_time)
                # print(review_time, review_date)  # 리뷰 시간을 출력

                # 리뷰 시간이 목표 날짜 이후인지 확인
                if review_date < target_date:
                    continue_loading = False
                    break
            except StaleElementReferenceException:
                continue  # 요소가 유효하지 않다면 다시 시도

        if continue_loading:
            # 더보기 버튼 클릭
            btn_more = wait_for_more_button()
            if btn_more:
                try:
                    driver.execute_script("arguments[0].scrollIntoView(true);", btn_more)
                    time.sleep(1)  # 추가 대기 시간
                    btn_more.click()
                    time.sleep(2)  # 추가 대기 시간
                except StaleElementReferenceException:
                    continue  # 요소가 유효하지 않다면 다시 시도
            else:
                break


# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# Chromedriver 경로 설정 (절대 경로 사용 권장)
chromedriver_path = os.path.abspath('./WebDriver/chromedriver.exe')

# Chromedriver 서비스 설정
service = Service(chromedriver_path)

# 크롬 드라이버 생성
driver = webdriver.Chrome(service=service, options=chrome_options)
# 브라우저 창 최대화
driver.maximize_window()
driver.get(url)

time.sleep(1)
# 입력 필드 찾기
input_field = driver.find_element(By.CSS_SELECTOR, "input[name='address_input']")
time.sleep(WAIT_TIME)
# 텍스트 지우기
input_field.clear()

# "덕성여자대학교" 텍스트 입력
input_field.send_keys("덕성여자대학교")

# 버튼 찾기 및 클릭
button = driver.find_element(By.CSS_SELECTOR, ".btn.btn-default.ico-pick")
button.click()

# 3번째 li 요소 찾기
third_li_element = WebDriverWait(driver, 1).until(
    EC.presence_of_element_located((By.XPATH, "(//ul[@class='dropdown-menu ng-scope am-flip-x bottom-left']//li)[3]/a"))
)

# 3번째 li 요소 클릭
third_li_element.click()

# 페이지 로드 후 일정 시간 대기
wait = WebDriverWait(driver, 1.5)

# ng-repeat을 가진 li 요소들 찾기 (대기 후)
li_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#category ul li")))
#print(li_elements)
# 원하는 인덱스의 li 요소들 선택 (4~9, 11)
selected_indices = [4, 5, 6, 7, 8, 10, 11]
selected_li_elements = [li_elements[i] for i in selected_indices]
#print(selected_li_elements)


# 각 선택된 li 요소 클릭
for index in selected_indices:
    try:
        # li 요소를 다시 찾아 클릭
        menu_category = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, f"#category ul li:nth-child({index + 1}) span")))
        menu_category_text = menu_category.text
        print(menu_category.text)
        menu_category.click()

        # 페이지 로드 후 일정 시간 대기
        time.sleep(WAIT_TIME)  # 드롭다운 로드를 위한 대기 시간 (필요 시 조절)

        # select 요소 찾기
        select_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".list-option-inner select")))
        # 드롭다운 클릭하여 열기
        select_element.click()
        # '리뷰 많은순' 옵션 선택
        review_count_option = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "option[value='review_count']")))
        review_count_option.click()

        # JavaScript로 페이지가 완전히 로드될 때까지 기다리기
        time.sleep(WAIT_TIME)  # 추가 대기 시간

        # print(f"Number of 'col-sm-6' divs for category index {index + 1}: {len(restaurant_name_list)}")
        for i in range(5):
            restaurant_name_list = driver.find_elements(By.CSS_SELECTOR, "div.restaurant-name")
            restaurant_name_list[i].click()
            time.sleep(1)

            # 페이지가 변경되었음을 확인하고 새로운 요소 찾기
            title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".restaurant-title")))
            storeName = title.find_element(By.XPATH, "span").text
            tab = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".restaurant-tab")))
            clean_review_tab = tab.find_element(By.XPATH, "//ul[@class='nav nav-tabs restaurant-tab']/li[2]/a")
            time.sleep(1)  # 추가 대기 시간
            clean_review_tab.click()

            # 목표 날짜 설정
            target_date = datetime(2023, 12, 31)

            # 목표 날짜까지 리뷰를 로드
            load_reviews_until_target_date(target_date)
            reviews = driver.find_elements(By.XPATH, "//li[@class='list-group-item star-point ng-scope']")
            for review in reviews:
                taste_point, quantity_point, delivery_point, order_items, review_text = extract_review_info(review)
                data = {
                    'menu_category': menu_category_text,  # menu_category가 어디에서 오는지에 따라 변경
                    'store_name': storeName,
                    'taste_point': taste_point,
                    'quantity_point': quantity_point,
                    'delivery_point': delivery_point,
                    'order_items': order_items,
                    'review_text': review_text
                }
                print(data)
                results.append(data)

                # 뒤로 가기
            driver.back()
            time.sleep(2)

    except Exception as e:
        print(f"Error clicking element at index {index}: {e}")
        time.sleep(2)
print(results)

# DataFrame으로 변환 및 CSV 파일로 저장
df = pd.DataFrame(results)
print(df)
df.to_csv('reviews.csv', index=False, encoding='utf-8-sig')
driver.quit()

## 2. 데이터 전처리



### 크롤링한 리뷰 데이터 로드

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
import pandas as pd
# 저장된 CSV 파일을 다시 읽어서 확인
review_df = pd.read_csv('reviews2.csv', encoding='utf-8-sig')

# DataFrame 확인
print(review_df.head())

  menu_category store_name  taste_point  quantity_point  delivery_point  \
0            치킨       닭치GO            5               5             NaN   
1            치킨       닭치GO            5               5             NaN   
2            치킨       닭치GO            5               5             NaN   
3            치킨       닭치GO            5               5             NaN   
4            치킨       닭치GO            5               5             NaN   

                                         order_items           review_text  
0  반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(후라이드),반반치킨 선택(양념치킨))        항상 맛있게 먹고있습니다~  
1  나 혼 자 순 살 반 반/1(나혼자 반반세트 치킨 선택(스노윙),나혼자 반반세트 치...           늘 잘 먹고있습니다!  
2  반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(후라이드),반반치킨 선택(양념치킨))  치킨생각나면 닭치Go가 생각나요 ㅎㅎ  
3  나 혼 자 순 살 반 반/1(나혼자 반반세트 치킨 선택(후라이드),나혼자 반반세트 ...          이집은 항상 맛집이에요  
4  반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(핫양념치킨),반반치킨 선택(허니...           맛있게잘먹었습니다~~  


In [ ]:
# DataFrame의 행과 열의 수 확인
print("\nDataFrame Shape:")
print(review_df.shape)


DataFrame Shape:
(11130, 7)


In [ ]:
review2_df = pd.read_csv('reviews_encoding2.csv', encoding='utf-8')

# DataFrame 확인
print(review2_df.head())

  menu_category store_name  taste_point  quantity_point  delivery_point  \
0            치킨       닭치GO            5               5             NaN   
1            치킨       닭치GO            5               5             NaN   
2            치킨       닭치GO            5               5             NaN   
3            치킨       닭치GO            5               5             NaN   
4            치킨       닭치GO            5               5             NaN   

                                         order_items           review_text  
0  반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(후라이드),반반치킨 선택(양념치킨))        항상 맛있게 먹고있습니다~  
1  나 혼 자 순 살 반 반/1(나혼자 반반세트 치킨 선택(스노윙),나혼자 반반세트 치...           늘 잘 먹고있습니다!  
2  반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(후라이드),반반치킨 선택(양념치킨))  치킨생각나면 닭치Go가 생각나요 ㅎㅎ  
3  나 혼 자 순 살 반 반/1(나혼자 반반세트 치킨 선택(후라이드),나혼자 반반세트 ...          이집은 항상 맛집이에요  
4  반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(핫양념치킨),반반치킨 선택(허니...           맛있게잘먹었습니다~~  


In [ ]:
# DataFrame의 행과 열의 수 확인
print("\nDataFrame Shape:")
print(review2_df.shape)


DataFrame Shape:
(6330, 7)


In [ ]:
review_df['menu_category'].unique()

array(['치킨', '피자/양식', '중국집', '한식', '일식/돈까스', '야식', '분식'], dtype=object)

In [ ]:
review2_df['menu_category'].unique()

array(['치킨', '피자/양식', '야식', '분식'], dtype=object)

selenium을 이용하여 동적 크롤링을 두 번 진행하였으나, 일부 카테고리는 크롤링이 안되었거나, '치킨'의 경우 중간에 크롤링을 하다 말고 에러가 생겨 다음 '피자/양식' 다음 카테고리로 넘어간 것을 발견.   
아래에서 두 개의 review_df 합치기 진행


-------------



### 두 개 review_df 합치기 전 `menu_category`, `store_name` 겹치는 데이터 제거

#### 두 개의 review df `치킨` 카테고리 확인

In [ ]:
# '치킨' 카테고리의 고유 값을 확인
chicken_unique_stores = review_df[review_df['menu_category'] == '치킨']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '치킨' category:")
print(chicken_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '치킨' category:")
print(len(chicken_unique_stores))

Unique stores in '치킨' category:
['닭치GO']

Number of unique stores in '치킨' category:
1


In [ ]:
# '치킨' 카테고리의 고유 값을 확인
chicken_unique_stores = review2_df[review2_df['menu_category'] == '치킨']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '치킨' category:")
print(chicken_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '치킨' category:")
print(len(chicken_unique_stores))

Unique stores in '치킨' category:
['닭치GO' '옆집떡볶이-미아점' '지코바치킨-서울수유1호점' '후라이드참잘하는집-쌍문점' '처갓집양념치킨-삼양점']

Number of unique stores in '치킨' category:
5


In [ ]:
print(len(review_df[(review_df['menu_category'] == '치킨') & (review_df['store_name'] == '닭치GO')]))

360


In [ ]:
len(review2_df[(review2_df['menu_category'] == '치킨') & (review2_df['store_name'] == '닭치GO')])

360

In [ ]:
# review2_df에 '치킨' 카테고리의 '닭치GO' 데이터 제거
review2_df = review2_df[~((review2_df['menu_category'] == '치킨') & (review2_df['store_name'] == '닭치GO'))]

In [ ]:
review2_df.shape

(5970, 7)

#### 두 개의 review df `피자/양식` 카테고리 확인


In [ ]:
# '피자/양식' 카테고리의 고유 값을 확인
pizza_unique_stores = review_df[review_df['menu_category'] == '피자/양식']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '피자/양식' category:")
print(pizza_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '피자/양식' category:")
print(len(pizza_unique_stores))

Unique stores in '피자/양식' category:
['피자글릭-강북점' '삿포로돈까스앤스파게티' '치탈리아-쌍문점' '청년피자-도봉구점' '두마리찜닭두찜-강북미아점']

Number of unique stores in '피자/양식' category:
5


In [ ]:
# '피자/양식' 카테고리의 고유 값을 확인
pizza_unique_stores = review2_df[review2_df['menu_category'] == '피자/양식']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '피자/양식' category:")
print(pizza_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '피자/양식' category:")
print(len(pizza_unique_stores))

Unique stores in '피자/양식' category:
['피자글릭-강북점' '삿포로돈까스앤스파게티']

Number of unique stores in '피자/양식' category:
2


In [ ]:
print('review_df 피자글릭-강북점 개수: ', len(review_df[(review_df['menu_category'] == '피자/양식') & (review_df['store_name'] == '피자글릭-강북점')]))
print('review2_df 피자글릭-강북점 개수: ',len(review2_df[(review2_df['menu_category'] == '피자/양식') & (review2_df['store_name'] == '피자글릭-강북점')]))

review_df 피자글릭-강북점 개수:  420
review2_df 피자글릭-강북점 개수:  420


In [ ]:
print('review_df 삿포로돈까스앤스파게티 개수: ', len(review_df[(review_df['menu_category'] == '피자/양식') & (review_df['store_name'] == '삿포로돈까스앤스파게티')]))
print('review2_df 삿포로돈까스앤스파게티 개수: ',len(review2_df[(review2_df['menu_category'] == '피자/양식') & (review2_df['store_name'] == '삿포로돈까스앤스파게티')]))

review_df 삿포로돈까스앤스파게티 개수:  340
review2_df 삿포로돈까스앤스파게티 개수:  340


In [ ]:
# review2_df '피자/양식' 카테고리의 '피자글릭-강북점'&삿포로돈까스앤스파게티' 데이터 제거
review2_df = review2_df[~((review2_df['menu_category'] == '피자/양식') & (review2_df['store_name'] == '피자글릭-강북점')) & ~((review2_df['menu_category'] == '피자/양식') & (review2_df['store_name'] == '삿포로돈까스앤스파게티'))]
print(review2_df.shape)

(5210, 7)


#### 두 개의 review df `야식` 카테고리 확인

In [ ]:
# '야식' 카테고리의 고유 값을 확인
yasik_unique_stores = review_df[review_df['menu_category'] == '야식']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '야식' category:")
print(yasik_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '야식' category:")
print(len(yasik_unique_stores))

Unique stores in '야식' category:
['족발야시장-수유역점' '옆집떡볶이-미아점']

Number of unique stores in '야식' category:
2


In [ ]:
# '야식' 카테고리의 고유 값을 확인
yasik_unique_stores = review2_df[review2_df['menu_category'] == '야식']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '야식' category:")
print(yasik_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '야식' category:")
print(len(yasik_unique_stores))

Unique stores in '야식' category:
['족발야시장-수유역점' '옆집떡볶이-미아점' '지코바치킨-서울수유1호점' '24시코리아홈푸드' '후라이드참잘하는집-쌍문점']

Number of unique stores in '야식' category:
5


In [ ]:
print('review_df 족발야시장-수유역점 개수: ', len(review_df[(review_df['menu_category'] == '야식') & (review_df['store_name'] == '족발야시장-수유역점')]))
print('review2_df 족발야시장-수유역점 개수: ',len(review2_df[(review2_df['menu_category'] == '야식') & (review2_df['store_name'] == '족발야시장-수유역점')]))

review_df 족발야시장-수유역점 개수:  440
review2_df 족발야시장-수유역점 개수:  430


In [ ]:
print('review_df 옆집떡볶이-미아점 개수: ', len(review_df[(review_df['menu_category'] == '야식') & (review_df['store_name'] == '옆집떡볶이-미아점')]))
print('review2_df 옆집떡볶이-미아점 개수: ',len(review2_df[(review2_df['menu_category'] == '야식') & (review2_df['store_name'] == '옆집떡볶이-미아점')]))

review_df 옆집떡볶이-미아점 개수:  790
review2_df 옆집떡볶이-미아점 개수:  790


In [ ]:
# review2_df '야식' 카테고리의 '옆집떡볶이-미아점' 데이터 제거
review2_df = review2_df[~((review2_df['menu_category'] == '야식') & (review2_df['store_name'] == '옆집떡볶이-미아점'))]
print(review2_df.shape)

(4420, 7)


In [ ]:
# review2_df '야식' 카테고리의 '족발야시장-수유역점' 데이터 제거
review2_df = review2_df[~((review2_df['menu_category'] == '야식') & (review2_df['store_name'] == '족발야시장-수유역점'))]
print(review2_df.shape)

(3990, 7)


#### 두 개의 review df `분식` 카테고리 확인

In [ ]:
# '분식' 카테고리의 고유 값을 확인
bunsik_unique_stores = review_df[review_df['menu_category'] == '분식']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '분식' category:")
print(bunsik_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '분식' category:")
print(len(bunsik_unique_stores))

Unique stores in '분식' category:
['옆집떡볶이-미아점' '24시코리아홈푸드' '맘스떡볶이n컵밥-수유점' '차고집냉면-강북점' '걸작떡볶이치킨-쌍문창동점']

Number of unique stores in '분식' category:
5


In [ ]:
# '분식' 카테고리의 고유 값을 확인
bunsik_unique_stores = review2_df[review2_df['menu_category'] == '분식']['store_name'].unique()

# 고유 값 출력
print("Unique stores in '분식' category:")
print(bunsik_unique_stores)

# 고유 항목 수 출력
print("\nNumber of unique stores in '분식' category:")
print(len(bunsik_unique_stores))

Unique stores in '분식' category:
['옆집떡볶이-미아점' '24시코리아홈푸드' '맘스떡볶이n컵밥-수유점' '차고집냉면-강북점' '걸작떡볶이치킨-쌍문창동점']

Number of unique stores in '분식' category:
5


In [ ]:
print('review_df 옆집떡볶이-미아점 개수: ', len(review_df[(review_df['menu_category'] == '분식') & (review_df['store_name'] == '옆집떡볶이-미아점')]))
print('review2_df 옆집떡볶이-미아점 개수: ',len(review2_df[(review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '옆집떡볶이-미아점')]))

review_df 옆집떡볶이-미아점 개수:  790
review2_df 옆집떡볶이-미아점 개수:  790


In [ ]:
print('review_df 24시코리아홈푸드 개수: ', len(review_df[(review_df['menu_category'] == '분식') & (review_df['store_name'] == '24시코리아홈푸드')]))
print('review2_df 24시코리아홈푸드 개수: ',len(review2_df[(review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '24시코리아홈푸드')]))

review_df 24시코리아홈푸드 개수:  100
review2_df 24시코리아홈푸드 개수:  100


In [ ]:
print('review_df 맘스떡볶이n컵밥-수유점 개수: ', len(review_df[(review_df['menu_category'] == '분식') & (review_df['store_name'] == '맘스떡볶이n컵밥-수유점')]))
print('review2_df 맘스떡볶이n컵밥-수유점 개수: ',len(review2_df[(review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '맘스떡볶이n컵밥-수유점')]))

review_df 맘스떡볶이n컵밥-수유점 개수:  380
review2_df 맘스떡볶이n컵밥-수유점 개수:  380


In [ ]:
print('review_df 차고집냉면-강북점 개수: ', len(review_df[(review_df['menu_category'] == '분식') & (review_df['store_name'] == '차고집냉면-강북점')]))
print('review2_df 차고집냉면-강북점 개수: ',len(review2_df[(review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '차고집냉면-강북점')]))

review_df 차고집냉면-강북점 개수:  280
review2_df 차고집냉면-강북점 개수:  280


In [ ]:
print('review_df 걸작떡볶이치킨-쌍문창동점 개수: ', len(review_df[(review_df['menu_category'] == '분식') & (review_df['store_name'] == '걸작떡볶이치킨-쌍문창동점')]))
print('review2_df 걸작떡볶이치킨-쌍문창동점 개수: ',len(review2_df[(review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '걸작떡볶이치킨-쌍문창동점')]))

review_df 걸작떡볶이치킨-쌍문창동점 개수:  90
review2_df 걸작떡볶이치킨-쌍문창동점 개수:  90


In [ ]:
# review2_df '분식' 카테고리의 '옆집떡볶이-미아점'&'24시코리아홈푸드'&'맘스떡볶이n컵밥-수유점'&'차고집냉면-강북점'&'걸작떡볶이치킨-쌍문창동점' 데이터 제거
review2_df = review2_df[~((review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '옆집떡볶이-미아점')) & ~((review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '24시코리아홈푸드'))
                      & ~((review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '맘스떡볶이n컵밥-수유점')) & ~((review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '차고집냉면-강북점'))
                         & ~((review2_df['menu_category'] == '분식') & (review2_df['store_name'] == '걸작떡볶이치킨-쌍문창동점'))]
print(review2_df.shape)

(2350, 7)


#### review_df, review2_df 전처리 후 데이터 개수 확인 & 합치기

In [ ]:
print(review_df.shape, review_df['menu_category'].unique())
print(review2_df.shape, review2_df['menu_category'].unique())

(11130, 7) ['치킨' '피자/양식' '중국집' '한식' '일식/돈까스' '야식' '분식']
(2350, 7) ['치킨' '야식']


In [ ]:
print(review_df[review_df['menu_category'] == '치킨']['store_name'].unique())
print(review2_df[review2_df['menu_category'] == '치킨']['store_name'].unique())

['닭치GO']
['옆집떡볶이-미아점' '지코바치킨-서울수유1호점' '후라이드참잘하는집-쌍문점' '처갓집양념치킨-삼양점']


In [ ]:
print(review_df[review_df['menu_category'] == '야식']['store_name'].unique())
print(review2_df[review2_df['menu_category'] == '야식']['store_name'].unique())

['족발야시장-수유역점' '옆집떡볶이-미아점']
['지코바치킨-서울수유1호점' '24시코리아홈푸드' '후라이드참잘하는집-쌍문점']


In [ ]:
# DataFrame 합치기
combined_df = pd.concat([review_df, review2_df], ignore_index=True)
combined_df.to_csv('combined_reviews.csv', index=False, encoding='utf-8-sig')

In [ ]:
combined_df.shape

(13480, 7)

In [ ]:
# menu_category별로 그룹화
grouped_dfs = {category: combined_df[combined_df['menu_category'] == category] for category in combined_df['menu_category'].unique()}

# 결과를 확인하기 위해 각 카테고리별 데이터프레임의 크기를 출력
for category, df in grouped_dfs.items():
    print(f"{category} : {df.shape}")

치킨 : (2050, 7)
피자/양식 : (1810, 7)
중국집 : (2200, 7)
한식 : (2520, 7)
일식/돈까스 : (1370, 7)
야식 : (1890, 7)
분식 : (1640, 7)


In [ ]:
# 카테고리별 가게명 확인
for category in combined_df['menu_category'].unique():
  print(category, combined_df[combined_df['menu_category'] == category]['store_name'].unique())

치킨 ['닭치GO' '옆집떡볶이-미아점' '지코바치킨-서울수유1호점' '후라이드참잘하는집-쌍문점' '처갓집양념치킨-삼양점']
피자/양식 ['피자글릭-강북점' '삿포로돈까스앤스파게티' '치탈리아-쌍문점' '청년피자-도봉구점' '두마리찜닭두찜-강북미아점']
중국집 ['라와마라탕-본점' '레드차이나' '차우린해물짬뽕' '챠이니-강북점' '최고성-쌍문점']
한식 ['옆집떡볶이-미아점' '24시코리아홈푸드' '라와마라탕-본점' '삿포로돈까스앤스파게티' '맘스떡볶이n컵밥-수유점']
일식/돈까스 ['24시코리아홈푸드' '삿포로돈까스앤스파게티' '차고집냉면-강북점' '다온초밥-본점' '육회한연어-수유점']
야식 ['족발야시장-수유역점' '옆집떡볶이-미아점' '지코바치킨-서울수유1호점' '24시코리아홈푸드' '후라이드참잘하는집-쌍문점']
분식 ['옆집떡볶이-미아점' '24시코리아홈푸드' '맘스떡볶이n컵밥-수유점' '차고집냉면-강북점' '걸작떡볶이치킨-쌍문창동점']


In [ ]:
# 각 store_name별로 속한 카테고리를 찾기 위해 빈 딕셔너리 생성
store_categories = {}
menu_categories = combined_df['menu_category'].unique()

# 각 카테고리별로 store_name을 추출하여 store_categories 딕셔너리에 추가
for category in menu_categories:
    category_df = combined_df[combined_df['menu_category'] == category]
    for store in category_df['store_name'].unique():
        if store not in store_categories:
            store_categories[store] = []
        store_categories[store].append(category)

# 겹치는 카테고리가 있는 store_name들을 찾기
overlapping_stores = {store: categories for store, categories in store_categories.items() if len(categories) > 1}

In [ ]:
overlapping_stores

{'옆집떡볶이-미아점': ['치킨', '한식', '야식', '분식'],
 '지코바치킨-서울수유1호점': ['치킨', '야식'],
 '후라이드참잘하는집-쌍문점': ['치킨', '야식'],
 '삿포로돈까스앤스파게티': ['피자/양식', '한식', '일식/돈까스'],
 '라와마라탕-본점': ['중국집', '한식'],
 '24시코리아홈푸드': ['한식', '일식/돈까스', '야식', '분식'],
 '맘스떡볶이n컵밥-수유점': ['한식', '분식'],
 '차고집냉면-강북점': ['일식/돈까스', '분식']}

In [ ]:
# 겹치는 카테고리가 있는 store_name들 단일화
store_specific_categories = {
    '옆집떡볶이-미아점': '분식',
    '지코바치킨-서울수유1호점': '치킨',
    '후라이드참잘하는집-쌍문점': '치킨',
    '삿포로돈까스앤스파게티': '일식/돈까스',
    '라와마라탕-본점': '중국집',
    '24시코리아홈푸드': '한식',
    '맘스떡볶이n컵밥-수유점': '한식',
    '차고집냉면-강북점': '분식'
}

In [ ]:
# 다른 카테고리에 있는 store_name 제거
for store, category in store_specific_categories.items():
    # 현재 store와 category가 매칭되지 않는 행을 제거
    combined_df = combined_df[~((combined_df['store_name'] == store) & (combined_df['menu_category'] != category))]

In [ ]:
# 카테고리별 가게명 확인
for category in combined_df['menu_category'].unique():
  print(category, combined_df[combined_df['menu_category'] == category]['store_name'].unique())

치킨 ['닭치GO' '지코바치킨-서울수유1호점' '후라이드참잘하는집-쌍문점' '처갓집양념치킨-삼양점']
피자/양식 ['피자글릭-강북점' '치탈리아-쌍문점' '청년피자-도봉구점' '두마리찜닭두찜-강북미아점']
중국집 ['라와마라탕-본점' '레드차이나' '차우린해물짬뽕' '챠이니-강북점' '최고성-쌍문점']
한식 ['24시코리아홈푸드' '맘스떡볶이n컵밥-수유점']
일식/돈까스 ['삿포로돈까스앤스파게티' '다온초밥-본점' '육회한연어-수유점']
야식 ['족발야시장-수유역점']
분식 ['옆집떡볶이-미아점' '차고집냉면-강북점' '걸작떡볶이치킨-쌍문창동점']


In [ ]:
# 각 store_name별로 속한 카테고리를 찾기 위해 빈 딕셔너리 생성
store_categories = {}
menu_categories = combined_df['menu_category'].unique()

# 각 카테고리별로 store_name을 추출하여 store_categories 딕셔너리에 추가
for category in menu_categories:
    category_df = combined_df[combined_df['menu_category'] == category]
    for store in category_df['store_name'].unique():
        if store not in store_categories:
            store_categories[store] = []
        store_categories[store].append(category)

# 겹치는 카테고리가 있는 store_name들을 찾기
overlapping_stores = {store: categories for store, categories in store_categories.items() if len(categories) > 1}
print(overlapping_stores)

{}


In [ ]:
# menu_category별로 그룹화
grouped_dfs = {category: combined_df[combined_df['menu_category'] == category] for category in combined_df['menu_category'].unique()}

# 결과를 확인하기 위해 각 카테고리별 데이터프레임의 크기를 출력
for category, df in grouped_dfs.items():
    print(f"{category} : {df.shape}")

print(combined_df.shape)

치킨 : (1260, 7)
피자/양식 : (1470, 7)
중국집 : (2200, 7)
한식 : (480, 7)
일식/돈까스 : (990, 7)
야식 : (440, 7)
분식 : (1160, 7)
(8000, 7)


In [ ]:
# csv로 저장
combined_df.to_csv('요기요리뷰_상호단일카테고리_전처리.csv', index=False, encoding='utf-8-sig')

In [ ]:
combined_df.reset_index(drop=True, inplace=True)

가게마다 단일 카테고리로 전처리 완료

### 리뷰 텍스트 전처리

In [ ]:
import pandas as pd

# CSV 파일을 읽어 DataFrame으로 변환
combined_df = pd.read_csv('요기요리뷰_상호단일카테고리_전처리.csv')

# DataFrame 출력
combined_df

,menu_category,store_name,taste_point,quantity_point,delivery_point,order_items,review_text,cleaned_review_text
0,치킨,닭치GO,5,5,NaN,"반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(후라이드),반반치킨 선택(양념치킨))",항상 맛있게 먹고있습니다~,항상 맛있게 먹고있습니다
1,치킨,닭치GO,5,5,NaN,"나 혼 자 순 살 반 반/1(나혼자 반반세트 치킨 선택(스노윙),나혼자 반반세트 치...",늘 잘 먹고있습니다!,늘 잘 먹고있습니다
2,치킨,닭치GO,5,5,NaN,"반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(후라이드),반반치킨 선택(양념치킨))",치킨생각나면 닭치Go가 생각나요 ㅎㅎ,치킨생각나면 가 생각나요
3,치킨,닭치GO,5,5,NaN,"나 혼 자 순 살 반 반/1(나혼자 반반세트 치킨 선택(후라이드),나혼자 반반세트 ...",이집은 항상 맛집이에요,이집은 항상 맛집이에요
4,치킨,닭치GO,5,5,NaN,"반반치킨/1(뼈／순살 선택(（순살）),반반치킨 선택(핫양념치킨),반반치킨 선택(허니...",맛있게잘먹었습니다~~,맛있게잘먹었습니다
...,...,...,...,...,...,...,...,...
7995,분식,걸작떡볶이치킨-쌍문창동점,5,5,5.0,"로제떡볶이R/1(떡볶이 맛 선택(1단계:달콤해서 아이들도 좋아해요),꿀조합 선택(선...",잘먹었습니다 감사합니다!,잘먹었습니다 감사합니다
7996,분식,걸작떡볶이치킨-쌍문창동점,5,5,5.0,"가래떡떡볶이/1(떡볶이 맛 선택(4단계:불닭볶음면이 맵지 않다면 추천 드려요),떡볶...",항상 맛있어서 잘 시켜먹고있아요~~~,항상 맛있어서 잘 시켜먹고있아요
7997,분식,걸작떡볶이치킨-쌍문창동점,5,5,5.0,"[추억의]짜장떡볶이R/1(맛선택（짜장떡볶이）(기본 （약간매운맛）),떡볶이 토핑 추가...",우삼겹은 제가 별로도 세팅해서 먹었는데 떡볶이가 참 맛있습니다.,은 제가 별로도 세팅해서 먹었는데 가 참 맛있습니다
7998,분식,걸작떡볶이치킨-쌍문창동점,4,3,5.0,"치즈떡볶이R/1(어묵많이 선택(어묵많이 변경),떡볶이 맛 선택(3단계:신라면보다 약...","맛있게잘먹었습ㄴ당, 배달엄청 빠르시네영, 깜놀! 번창하세영,",맛있게잘먹었습당 배달엄청 빠르시네영 깜놀 번창하세영


#### 1. 불필요한 텍스트 정제 (`이모지, 특수문자, 영어 표현, 숫자`)

In [ ]:
combined_df['review_text']

0                            항상 맛있게 먹고있습니다~
1                               늘 잘 먹고있습니다!
2                      치킨생각나면 닭치Go가 생각나요 ㅎㅎ
3                              이집은 항상 맛집이에요
4                               맛있게잘먹었습니다~~
                       ...                 
7995                          잘먹었습니다 감사합니다!
7996                   항상 맛있어서 잘 시켜먹고있아요~~~
7997    우삼겹은 제가 별로도 세팅해서 먹었는데 떡볶이가 참 맛있습니다.
7998      맛있게잘먹었습ㄴ당, 배달엄청 빠르시네영, 깜놀! 번창하세영,
7999      오랜만에 맛있게 잘 먹었어요 근데 이제 쿠폰은 없어진건가요?
Name: review_text, Length: 8000, dtype: object

In [ ]:
import re

In [ ]:
def remove_special_characters_and_emojis(text):
  # 이모지 제거
  emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # 이모티콘
        u"\U0001F300-\U0001F5FF"  # 기호 & 픽토그램
        u"\U0001F680-\U0001F6FF"  # 교통 & 지도 기호
        u"\U0001F700-\U0001F77F"  # 추가 이모티콘
        u"\U0001F780-\U0001F7FF"  # 추가 이모티콘
        u"\U0001F800-\U0001F8FF"  # 추가 이모티콘
        u"\U0001F900-\U0001F9FF"  # 추가 이모티콘
        u"\U0001FA00-\U0001FA6F"  # 추가 이모티콘
        u"\U0001FA70-\U0001FAFF"  # 추가 이모티콘
        u"\U00002600-\U000026FF"  # 다양한 기호
        u"\U00002700-\U000027BF"  # 다양한 기호
        "]+", flags=re.UNICODE
  )
  text = emoji_pattern.sub(r'', text)

  # 영어 표현, 숫자, 특수 문자 제거
  text = re.sub(r'[^a-zA-Z가-힣0-9\s]', '', text)
  return text

combined_df['cleaned_review_text'] = combined_df['review_text'].apply(remove_special_characters_and_emojis)
print(combined_df[['review_text', 'cleaned_review_text']])

                              review_text                 cleaned_review_text
0                          항상 맛있게 먹고있습니다~                       항상 맛있게 먹고있습니다
1                             늘 잘 먹고있습니다!                          늘 잘 먹고있습니다
2                    치킨생각나면 닭치Go가 생각나요 ㅎㅎ                  치킨생각나면 닭치Go가 생각나요 
3                            이집은 항상 맛집이에요                        이집은 항상 맛집이에요
4                             맛있게잘먹었습니다~~                           맛있게잘먹었습니다
...                                   ...                                 ...
7995                        잘먹었습니다 감사합니다!                        잘먹었습니다 감사합니다
7996                 항상 맛있어서 잘 시켜먹고있아요~~~                   항상 맛있어서 잘 시켜먹고있아요
7997  우삼겹은 제가 별로도 세팅해서 먹었는데 떡볶이가 참 맛있습니다.  우삼겹은 제가 별로도 세팅해서 먹었는데 떡볶이가 참 맛있습니다
7998    맛있게잘먹었습ㄴ당, 배달엄청 빠르시네영, 깜놀! 번창하세영,        맛있게잘먹었습당 배달엄청 빠르시네영 깜놀 번창하세영
7999    오랜만에 맛있게 잘 먹었어요 근데 이제 쿠폰은 없어진건가요?    오랜만에 맛있게 잘 먹었어요 근데 이제 쿠폰은 없어진건가요

[8000 rows x 2 columns]


#### 2. null값 제거

In [ ]:
new_df = combined_df[['review_text', 'cleaned_review_text']]

In [ ]:
# null 값 확인
null_counts = new_df['cleaned_review_text'].isnull().sum()
print("각 열별 null 값 개수:")
print(null_counts)

# 전체 데이터프레임에서 null 값이 있는 행 확인
null_data = new_df[new_df.isnull().any(axis=1)]
print("\nnull 값이 있는 행:")
print(null_data)

각 열별 null 값 개수:
0

null 값이 있는 행:
Empty DataFrame
Columns: [review_text, cleaned_review_text]
Index: []


In [ ]:
# csv로 저장
combined_df.to_csv('리뷰_전처리.csv', index=False, encoding='utf-8-sig')

## 3. 문장 생성 딥러닝 모델 학습   

모델이 너무 커 colab에서 학습을 시작하면 세션이 나감. 학습은 로컬에서 진행

### 커스텀 데이터셋 정의

In [ ]:
# custom_dataset.py
import torch
from torch.utils.data import Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=50):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': input_ids
        }

### 모델 학습

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import re
from custom_dataset import CustomDataset

if __name__ == '__main__':
    # CSV 파일을 읽어 DataFrame으로 변환
    combined_df = pd.read_csv('리뷰_전처리.csv')
    print(combined_df[['review_text', 'cleaned_review_text']])

    # Set CUDA_LAUNCH_BLOCKING for debugging
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
    model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")

    # Add pad token if necessary
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})  # Use EOS token as padding token if not set
        model.resize_token_embeddings(len(tokenizer))

    # Training settings
    train_texts = combined_df['cleaned_review_text'].tolist()
    train_dataset = CustomDataset(train_texts, tokenizer)
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)  # Set num_workers to 0

    optimizer = AdamW(model.parameters(), lr=1e-5)
    epochs = 10
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Early stopping settings
    early_stopping_patience = 3
    best_loss = float('inf')
    patience_counter = 0

    # Training loop
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    for epoch in range(epochs):
        epoch_loss = 0
        batch_count = 0
        for batch in train_dataloader:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).long()  # Ensure labels are of type torch.long

            # Debugging: Print input IDs and their range
            print(f"Input IDs: {input_ids}")
            print(f"Max Input ID: {input_ids.max().item()}, Vocab Size: {tokenizer.vocab_size}")
            batch_count += 64
            print("batch count: ", batch_count, "epoch: ", epoch)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1} Loss: {avg_epoch_loss}")

        # Early stopping check
        if avg_epoch_loss < best_loss:
            best_loss = avg_epoch_loss
            patience_counter = 0
            # 모델 저장
            model.save_pretrained('best_model')
            tokenizer.save_pretrained('best_tokenizer')
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch + 1}")
                break

    # Save model and tokenizer
    model.save_pretrained('path_to_save_model')
    tokenizer.save_pretrained('path_to_save_tokenizer')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Input IDs: tensor([[ 9673,  7235, 10307,  ..., 51200, 51200, 51200],
        [28005,  7235, 29578,  ..., 51200, 51200, 51200],
        [11355, 18479, 10623,  ..., 51200, 51200, 51200],
        ...,
        [11355, 49067,  8084,  ..., 51200, 51200, 51200],
        [28005,  7235, 29578,  ..., 51200, 51200, 51200],
        [10997,  9168,  8236,  ..., 51200, 51200, 51200]])
Max Input ID: 51200, Vocab Size: 51200


## 4. 학습한 모델 로드

### 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 모델 및 토크나이저 경로
tokenizer_path = "/content/drive/MyDrive/Classroom/빅데이터시스템-01/과제/path_to_save_tokenizer"
model_path = "/content/drive/MyDrive/Classroom/빅데이터시스템-01/과제/path_to_save_model"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

### 문장 생성

In [ ]:
def generate_text(prompt, max_length=50):
    # Encode the prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

In [ ]:
# Example usage
prompt = "오늘은"
generated_text = generate_text(prompt, max_length=50)
print(generated_text)

오늘은 좀 덜 매웠어요 그래도 맛있게 잘 먹었습니다 오늘도 감사합니다  맛있습니당  맵찔이라서 퍽퍽한데 쫄깃해서 좋았어용


In [ ]:
# Example usage
prompt = "배달"
generated_text = generate_text(prompt, max_length=50)
print(generated_text)

배달도 빠르고 맛있어요  서비스도 감사합니다 서비스로 주신 막국수도 맛있게 잘 먹었습니다 짬뽕국물 감사해요 짱 감사드려요 감사드립니당


In [ ]:
# Example usage
prompt = "하.."
generated_text = generate_text(prompt, max_length=50)
print(generated_text)

하.. 맛있어요  서비스 감사합니다 서비스로 주신 떡사리도 감사히 잘 먹었습니다   번창하세요  감사해요 번창하시고 번창 하세여 감사드려요
